## Preprocessing

Estraiamo un campione di frasi a partire dai file conllu di Wikipedia italiana elaborata. 

In [1]:
import os
import re
import pandas as pd
from tqdm import tqdm


In [2]:
def get_id(line):
    sent_id = re.sub(r'\D', '', line)
    return sent_id

def get_text(line):
    sent = line[9:].rstrip('\n')
    return sent

def get_sentences(file_conllu):
    sentences = []
    ids = []
    with tqdm(total=sum(1 for _ in open(file_conllu, 'r', encoding='utf-8')), desc='Progresso estrazione frasi del file {file_conllu}') as pbar:
        for line in open(file_conllu, 'r', encoding='utf-8'):
            pbar.update(1)  # Aggiorna la barra di avanzamento
            if line.startswith("# sent_id"):
                current_id = get_id(line)  
                ids.append(current_id)
            elif line.startswith("# text"):
                current_sent = get_text(line)
                sentences.append(current_sent)
    return ids, sentences

In [3]:
#nel caso in cui dovessimo itereare sui diversi file nella dir
ds_directory = "C:/Users/bergo/OneDrive - University of Pisa/Tesi Magistrale/wiki_conllu"
ds_files = []
for file_name in os.listdir(ds_directory):
    file_path = os.path.join(ds_directory, file_name)
    ds_files.append(file_path)  #ottengo il nome dei diversi file all'interno della directory
print(ds_files)

['C:/Users/bergo/OneDrive - University of Pisa/Tesi Magistrale/wiki_conllu\\wiki_10_2021_015.conllu']


In [4]:
'''with open(ds_files[0], "r", encoding="utf-8") as file:
    # Legge tutte le righe del file in una lista
    righe = file.readlines()'''

'with open(ds_files[0], "r", encoding="utf-8") as file:\n    # Legge tutte le righe del file in una lista\n    righe = file.readlines()'

In [5]:
data_df = pd.DataFrame(columns=["id", "text"])
data_df["id"], data_df["text"] = get_sentences(ds_files[0])   #per ora lavoro su un singolo file

Progresso estrazione frasi del file {file_conllu}: 100%|██████████| 29437395/29437395 [00:30<00:00, 966186.88it/s] 


In [6]:
data_df.head()

,id,text
0,1,impeto.
1,2,L’occupazione di Pastrengo.
2,3,Oramai per gli austriaci si trattava di prende...
3,4,Gli uomini della Brigata “Cuneo” intanto entra...
4,5,Ma una batteria di artiglieria a cavallo dei p...


In [7]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1057516 entries, 0 to 1057515
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   id      1057516 non-null  object
 1   text    1057516 non-null  object
dtypes: object(2)
memory usage: 16.1+ MB


In [9]:
samplino = data_df.sample(n=4000, random_state=42) #campione piccino per fare le prove

### Estrazione degli indici di complessità per ogni frase

Utilizziamo l'API _read-it_ 

In [10]:
import requests
import sys

In [14]:

SERVER_PATH = "http://api.italianlp.it"
#SERVER_PATH = "http://itanlp-gpu.ilc.cnr.it:13000"

def load_document(text):
    r = requests.post(SERVER_PATH + '/documents/',           # carica il documento nel database del server
                      data={'text': text,                    # durante il caricamento viene eseguita un'analisi linguistica necessaria per calcolare la leggibilita'
                          'lang': 'IT',
                          'extra_tasks': ["readability"]     # chiede al server di calcolare anche la leggibilità del docuemnto
                  })
    doc_id = r.json()['id']                                  # id del documento nel database del server, che serve per richiedere i risultati delle analisi
    return doc_id

def get_doc_score(doc_id):
    r = requests.get(SERVER_PATH + '/documents/details/%s' % doc_id)
    result = r.json()
    return result



In [18]:
text = "Ho tanta voglia di pollo del mercato (quello ruspante), non vedo l'ora sia sabato per mangiarlo. Ogni tanto mi chiedo se ne vale la pena spendere 10 euro per mezzo pollo e delle patatine, però quando lo mangio mi sento felice."

doc_id = load_document(text)
r_score = get_doc_score(doc_id)

print(r_score)

{'named_entity_executed': False, 'doc_time': '2024-05-13T15:57:01.643925Z', 'readability_executed': True, 'sentences': {'count': 2, 'prev': None, 'data': [{'sentiment_positive_negative_probability': None, 'sentiment_value': None, 'sequence': 1, 'readability_score_all': 89.3436804643733, 'sentiment_negative_probability': None, 'sentiment_neutral_probability': None, 'tokens': [{'word': 'Ho', 'ten': 'p', 'sequence': 1, 'pos': 'V', 'per': '1', 'lemma': 'avere', 'num': 's', 'named_entity_instance': None, 'dep_type': 'ROOT', 'dep_parent_sequence': None, 'cpos': 'V', 'gen': None, 'mod': 'i'}, {'word': 'tanta', 'ten': None, 'sequence': 2, 'pos': 'DI', 'per': None, 'lemma': 'tanto', 'num': 's', 'named_entity_instance': None, 'dep_type': 'mod', 'dep_parent_sequence': 3, 'cpos': 'D', 'gen': 'f', 'mod': None}, {'word': 'voglia', 'ten': None, 'sequence': 3, 'pos': 'S', 'per': None, 'lemma': 'voglia', 'num': 's', 'named_entity_instance': None, 'dep_type': 'obj', 'dep_parent_sequence': 1, 'cpos': 'S'

In [19]:
print(r_score["sentences"]["data"][0]["raw_text"])

Ho tanta voglia di pollo del mercato (quello ruspante), non vedo l'ora sia sabato per mangiarlo.


In [20]:
#qui iteriamo sui risultati ottenuti per un singolo documento
def get_sen_scores(doc_scores):
    all_sent_rscore = []
    all_sent_text = []
    for i, sent_results in enumerate(doc_scores['sentences']['data']):
        all_sent_rscore = all_sent_rscore + sent_results['readability_score_all']
        all_sent_text = all_sent_text + sent_results['raw_text']
    return all_sent_text, all_sent_rscore

#si passa all'api un doc di 1000 frasi per volta.
def readability_extraction(sen_list):
    all_scores = []
    for i in range(0, len(sen_list), 1000):
        doc = "\n".join(sen_list[i:i+1000])
        doc_id = load_document(doc)
        r_scores = get_doc_score(doc_id)
        all_scores.append(r_scores)
    return all_scores

#qui si ottiene la struttura dati finale dove per ogni frase abbiamo lo score ottenuto da read_it
def get_final_data(all_scores):
    text_list = []
    score_list = []
    for doc_scores in all_scores:
        text, score = get_sen_scores(doc_scores)
        text_list.append(text)
        score_list.append(score)
    return text_list, score_list


In [21]:
samplino

,id,text
102389,102390,la voce del narratore fa posto a quella dei pe...
530508,530509,Ricardo Bofill nel Taller de Arquitectura usa ...
56620,56621,La denominazione del Gruppo passò da CB a CBOC.
240780,240781,Al traguardo 160 ciclisti portarono a termine ...
860993,860994,"In questa, la fanciulla scappa indossando il m..."
...,...,...
577044,577045,egli dovrà recarsi ad Aquileia allo scopo di o...
232398,232399,L'impulso origina nel fascio di His e va ai ve...
877414,877415,"La voce «""Bene qui làtuit, bene vixit""» (massi..."
707559,707560,Callen sostiene anche di parlare ceco e rumeno.


In [ ]:
#all_scores = readability_extraction(samplino["text"].tolist())
